In [107]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error
#reading the data
data = pd.read_csv('healthcare-dataset-stroke-data.csv')

In [112]:
data.tail()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
5105,18234,1,80.0,1,0,1,Private,Urban,83.75,unknown,never smoked,0
5106,44873,1,81.0,0,0,1,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,1,35.0,0,0,1,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,0,51.0,0,0,1,Private,Rural,166.29,25.6,formerly smoked,0
5109,44679,1,44.0,0,0,1,Govt_job,Urban,85.28,26.2,Unknown,0


In [109]:
#checking the datas for NaN values
data.age.isnull().value_counts()
data.gender.isnull().value_counts()
data.age.isnull().value_counts()
data.smoking_status.isnull().value_counts()
data.avg_glucose_level.isnull().value_counts()
data.Residence_type.isnull().value_counts()
data.work_type.isnull().value_counts()
data.heart_disease.isnull().value_counts()
data.bmi.isnull().value_counts()

bmi
False    4909
True      201
Name: count, dtype: int64

In [110]:
data.bmi = data.bmi.fillna('unknown')
data.gender = data.gender.replace('Male', 0).replace('Female', 1)
data.ever_married = data.ever_married.replace('Yes', 1)
data.ever_married = data.ever_married.replace('No', 0)

C:\Users\hwang\AppData\Local\Temp\ipykernel_36348\4151752887.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.ever_married = data.ever_married.replace('No', 0)


In [111]:
data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,0,67.0,0,1,1,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,1,61.0,0,0,1,Self-employed,Rural,202.21,unknown,never smoked,1
2,31112,0,80.0,0,1,1,Private,Rural,105.92,32.5,never smoked,1
3,60182,1,49.0,0,0,1,Private,Urban,171.23,34.4,smokes,1
4,1665,1,79.0,1,0,1,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,1,80.0,1,0,1,Private,Urban,83.75,unknown,never smoked,0
5106,44873,1,81.0,0,0,1,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,1,35.0,0,0,1,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,0,51.0,0,0,1,Private,Rural,166.29,25.6,formerly smoked,0


In [46]:
factors = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status'] 
x = data[factors]
y = data.stroke

In [47]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(x,y)


ValueError: could not convert string to float: 'Female'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns